In [1]:
import numpy as np
np.random.seed(1337)
from keras.datasets import mnist #keras本来就就有的数据集
from keras.models import Sequential  #keras的两种模型之一
from keras.layers import Dense, Dropout, Activation, Flatten  
from keras.utils import np_utils  
from keras import backend as K  
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
batch_size=128#每次训练样本个数,太小会训练慢，过拟合，太大会欠拟合
nb_classes=10#一共分成10类
epochs=12#12次迭代
img_rows,img_cols=28,28#图片大小，是灰度图
nb_filter=32#卷积核个数
kernel_size=(3,3)#卷积核大小
pool_size=(2,2)#池化大小

In [3]:
(X_train,y_train),(X_test,y_test)=mnist.load_data()

#### 'th'模式，即Theano模式会把100张RGB三通道的16×32（高为16宽为32）彩色图表示为下面这种形式（100,3,16,32），Caffe采取的也是这种方式。第0个维度是样本维，代表样本的数目，第1个维度是通道维，代表颜色通道数。后面两个就是高和宽了。而TensorFlow，即'tf'模式的表达形式是（100,16,32,3），即把通道维放在了最后。

In [4]:
#keras有两种backend,theano和tensorflow
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [5]:
X_train = X_train.astype('float32')  
X_test = X_test.astype('float32')  
#归一化
X_train /= 255  
X_test /= 255  
print('X_train shape:', X_train.shape)  
print(X_train.shape[0], 'train samples')  
print(X_test.shape[0], 'test samples')  

('X_train shape:', (60000, 28, 28, 1))
(60000, 'train samples')
(10000, 'test samples')


In [6]:
X_train[

SyntaxError: unexpected EOF while parsing (<ipython-input-6-6e15aefdac4f>, line 1)

In [7]:
# 转换为one_hot类型 ,标签为1则是0100000000，2是0010000000
Y_train = np_utils.to_categorical(y_train, nb_classes)  
Y_test = np_utils.to_categorical(y_test, nb_classes)  

In [8]:
Y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [9]:
Y_train[0].size

10

In [10]:
model = Sequential()#sequential模型可以随意插入神经网络层

##### 二维卷积层对二维输入进行滑动窗卷积，当使用该层作为第一层时，应提供input_shape参数。

filters：卷积核的数目；

kernel_size：卷积核的尺寸；

strides：卷积核移动的步长，分为行方向和列方向；

padding：边界模式，有“valid”，“same”或“full”，full需要以theano为后端；



In [12]:
# 卷积层1 参数是卷积核个数，卷积核大小，边界模式，输入（第一层要有）
model.add(Conv2D(nb_filter, (kernel_size[0], kernel_size[1]),  
                        padding='same',  
                        input_shape=input_shape)) # 卷积层1  

##### 激活层对一个层的输出施加激活函数。

预定义激活函数：

softmax，softplus，softsign，relu，tanh，sigmoid，hard_sigmoid，linear等。



In [13]:
#激活层，第一个conv层后的激活
model.add(Activation('relu')) 

In [15]:
model.add(Conv2D(nb_filter, (kernel_size[0], kernel_size[1]))) #卷积层2  
model.add(Activation('relu')) #激活层 

##### 池化层
pool_size：池化核尺寸；

strides：池化核移动步长；

padding：边界模式，有“valid”，“same”或“full”，full需要以theano为后端；

In [16]:
model.add(MaxPooling2D(pool_size=pool_size)) #池化层 

##### Dropout将在训练过程中每次更新参数时随机断开一定百分比（p）的输入神经元连接，Dropout层用于防止过拟合

In [17]:
model.add(Dropout(0.25)) #神经元随机失活  

##### Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。Flatten不影响batch的大小。

In [18]:
model.add(Flatten()) #拉成一维数据  

##### 输出单元的数量，即全连接层神经元的数量，作为第一层的Dense层必须指定input_shape
http://keras-cn.readthedocs.io/en/latest/layers/core_layer/

In [19]:
model.add(Dense(128)) #全连接层1  
model.add(Activation('relu')) #激活层  

In [20]:
model.add(Dropout(0.5)) #随机失活  

In [21]:
model.add(Dense(nb_classes)) #全连接层2  

In [22]:
model.add(Activation('softmax')) #Softmax评分 

##### 编译用来配置模型的学习过程，其参数有：

optimizer：字符串（预定义优化器名）或优化器对象；

loss：字符串（预定义损失函数名）或目标函数；

metrics：列表，包含评估模型在训练和测试时的网络性能的指标，典型用法是metrics=['accuracy']；

In [23]:
#编译模型
model.compile(loss='categorical_crossentropy',  
              optimizer='adadelta',  
              metrics=['accuracy'])  

In [24]:
#训练模型
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,  
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 161s - loss: 0.3870 - acc: 0.8822 - val_loss: 0.1010 - val_acc: 0.9684
Epoch 2/12
60000/60000 [==============================] - 162s - loss: 0.1460 - acc: 0.9567 - val_loss: 0.0640 - val_acc: 0.9791
Epoch 3/12
60000/60000 [==============================] - 160s - loss: 0.1084 - acc: 0.9680 - val_loss: 0.0537 - val_acc: 0.9822
Epoch 4/12
60000/60000 [==============================] - 159s - loss: 0.0903 - acc: 0.9731 - val_loss: 0.0446 - val_acc: 0.9856
Epoch 5/12
60000/60000 [==============================] - 161s - loss: 0.0808 - acc: 0.9757 - val_loss: 0.0402 - val_acc: 0.9870
Epoch 6/12
60000/60000 [==============================] - 199s - loss: 0.0737 - acc: 0.9782 - val_loss: 0.0387 - val_acc: 0.9878
Epoch 7/12
60000/60000 [==============================] - 198s - loss: 0.0677 - acc: 0.9805 - val_loss: 0.0351 - val_acc: 0.9884
Epoch 8/12
60000/60000 [=======================

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)  
print('Test score:', score[0])  
print('Test accuracy:', score[1])  